In [1]:
import sys
sys.path.append('../')
sys.path.append('../src')

from src.deepqlearning import DeepQlearning, NeuralNetwork
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'base_model'

# Set up agent

In [ ]:
class Player:
    def __init__(self, get_action) -> None:
        self.get_action = get_action

# Set up training

In [ ]:
Q = DeepQlearning(model=NeuralNetwork())
Q.training(n_training_game=100)
Q.play()

# Analyze training

In [ ]:
Q.plot_training_stats()
plt.show()

# Look at a game

In [ ]:
Q.play()

In [ ]:
player = Player(get_action=lambda state: Q.get_action(state))